# 12-Finding Protein Folding by Quantum Annealing

2012年に、〜がタンパク質折り畳み問題を量子アニーリングを用いて解きました[1]。

このチュートリアルでは、SAを用いてタンパク質折り畳み問題を解いてみました。

## 問題の概要とHPモデル、Mijazawa-Jernigan(MJ)モデル 

### 問題概要

タンパク質の立体構造をラティスフォールディングモデル(https://en.wikipedia.org/wiki/Lattice_protein)という単純な格子モデルとして捉え、HUBOの形式にエンコードし、最適解を求め、立体構造を決定します。特に参考論文[1]では、２次元上の構造問題として見做し、解いています。ラティスフォールディングモデルとして、タンパク質の構造を予測するために、HPモデルを使用しています。


### HPモデル 
- アミノ酸をＨ（疎水性、非極性アミノ酸）とＰ(親水性、極性アミノ酸）のいずれかに分ける。
- 参考：http://www.iba.t.u-tokyo.ac.jp/iba/AI/HP.pdf

### MJモデル
- 今回は単純化されたMJモデルを利用します。

<div>
    <img src="images/hp_model1.png" width="300" height="200">
    図１：ラティスフォールディングモデル模式図(wikipeadia)
</div>

## MJモデルのHUBOへの適用

用意された塩基列を特定の方向に回転させる操作をイジングモデルに対応させています。

今回使用する塩基列は、PSVKMAの配列で下記のように特定の塩基列が隣接すると安定状態になり、エネルギーが減ります。

このエネルギーの安定化を使っってコスト関数を最小化させることを考えます。また、今回塩基列を全て一度に処理するのは難しいのでいくつかのパターンに分けます。

すでにいくつかの折り畳まれたパターンから出発して安定状態を求めます。
数が多くないので書き出すことができ、最も低いエネルギー安定状態を求めることができます。

本チュートリアルでは、実験３のschemeに絞り、取り上げて一番エネルギーの低い状態を立式から導出したいと思います。

## 　コスト関数

タンパク質の構造をラティスフォールディングモデルとして考えるにあたり、一般のコスト関数は、以下となります。

$$E_p = E_{onsite} + E_{pw} + E_{ext}$$

- 第１項: タンパク質の塩基列が重ならない
- 第２項: は塩基列同士の近接の相互作用のエネルギー
- 第３項: 外部からの影響

論文中の実験３については、第３項は使用しないため、コスト関数は以下となります。

$$E_p = E_{onsite} + E_{pw}$$


## モデル

本チュートリアルでは、図　のScheme3のExperiment3を初期状態にして、考えたいと思います。塩基列は、

P-S-V-K-M-A

この順番ですが、今回はある程度折りたたまれた状態で始まります。


<div>
    <img src="images/scheme.png" width="600" height="600">
     図　：　（[1]のFigure を改変）
</div>

この場合、PSKVまでは決まっていて、Mは下か左が決まっていますので、Pから順番に書いてみると、

$$010010q_10q_2q_3$$

となり、３量子ビットの式に還元されます。$01$は右、$00$は下、$10$は左、Mは下か左しかないので、$00$か$10$なので、$0$は決まっていて、残りの$q_1$から$q_3$をイジング問題で求めます。

<div>
    <img src="images/make_costfunction_about_protein_folding.png" width="600" height="400">
    図　：　([1]のFigure を一部利用)
</div>


コスト関数は、

$$E = -1-4q_3+9q_1q_3+9q_2q_3-16q_1q_2q_3$$

## HUBOの直接解法

In [1]:
import openjij as oj

# Only SASampler can handle HUBO.
sampler = oj.SASampler()

# define polynomial
polynomial = {():-1, (3,): -4, (1,3): 9, (2,3): 9, (1,2,3): -16}

In [21]:
import numpy as np

response = sampler.sample_hubo(polynomial, vartype="BINARY", num_reads=100)
(np.array(response.energies)==-5)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False,  True, False,
       False, False, False, False, False,  True, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False,  True, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False,  True, False,
        True, False, False, False,  True,  True, False, False, False,
       False, False, False, False, False,  True, False,  True, False,
       False, False, False,  True, False, False, False, False,  True,
       False])

In [22]:

response = sampler.sample_hubo(polynomial, vartype="BINARY", updater="single spin flip", num_reads=100)
(np.array(response.energies)==-5)


array([False,  True,  True, False, False, False,  True,  True, False,
        True,  True, False,  True,  True, False, False, False,  True,
       False,  True,  True, False, False, False, False, False,  True,
       False,  True,  True, False, False,  True,  True,  True,  True,
        True, False,  True,  True, False,  True, False, False, False,
       False, False,  True, False, False, False,  True,  True,  True,
        True, False, False, False, False, False, False, False,  True,
        True,  True, False,  True,  True,  True, False, False, False,
        True, False, False,  True, False, False, False, False, False,
        True, False, False, False, False, False,  True,  True,  True,
       False, False, False,  True,  True,  True, False,  True, False,
       False])

## QUBO変換による解法

In [3]:
import dimod

# Polynomial相互作用、ペナルティの大きさ、変数のタイプを指定して対応するquadraticモデルを生成する。
bqm = dimod.make_quadratic(poly=polynomial, strength=5.0, vartype="BINARY")
print('0次の項:', bqm.offset)
print('1次の項:', dict(bqm.linear))    # bqm.linearはpythonのdictに変換して表示する。
print('2次の項:', dict(bqm.quadratic)) # bqm.quadraticもpythonのdictに変換して表示する。

0次の項: -1.0
1次の項: {3: -4.0, 1: 0.0, 2: 0.0, '1*2': 15.0}
2次の項: {(1, 3): 9.0, (2, 3): 9.0, (2, 1): 5.0, ('1*2', 3): -16.0, ('1*2', 1): -10.0, ('1*2', 2): -10.0}


In [4]:
# インデックスを1始まりの整数に変換する関数
def relabel_variables_as_integers(dimod_bqm):
    mapping = {}
    variables = list(dimod_bqm.variables)
    count = 1
    for key in variables:
        mapping[key] = count
        count += 1
    linear    = {mapping[k]:v for k,v in dimod_bqm.linear.items()}
    quadratic = {(mapping[k[0]], mapping[k[1]]):v for k,v in dimod_bqm.quadratic.items()}
    return dimod.BinaryQuadraticModel(linear, quadratic, dimod_bqm.offset, dimod_bqm.vartype), mapping


bqm_relabeled, mapping = relabel_variables_as_integers(bqm) # インデックスを1始まりに変換する。

print('0次の項:', bqm_relabeled.offset)
print('1次の項:', dict(bqm_relabeled.linear))    # bqm.linearはpythonのdictに変換して表示する。
print('2次の項:', dict(bqm_relabeled.quadratic)) # bqm.quadraticもpythonのdictに変換して表示する。
print('変数の対応関係:', mapping) # Relabelした後のインデックスと元のインデックスの対応関係を表示する。

0次の項: -1.0
1次の項: {2: 0.0, 1: -4.0, 3: 0.0, 4: 15.0}
2次の項: {(1, 2): 9.0, (3, 2): 5.0, (3, 1): 9.0, (4, 2): -10.0, (4, 1): -16.0, (4, 3): -10.0}
変数の対応関係: {3: 1, 1: 2, 2: 3, '1*2': 4}


In [5]:
# dimodのbqm_relabeledはOpenJijのBinaryQuadraticModelに変換する。
bqm_oj = oj.BinaryQuadraticModel(dict(bqm_relabeled.linear), dict(bqm_relabeled.quadratic), bqm_relabeled.offset, vartype="BINARY")

#　ここまでの前処理をした後に、sampleメソッドに投げることができる。
response = sampler.sample(bqm_oj)
print(response)

   1  2  3  4 energy num_oc.
0  1  0  1  1   -7.0       1
['BINARY', 1 rows, 1 samples, 4 variables]


In [6]:
# 元のHUBOの解に焼き直す。
hubo_configuration = {i: response.record[0][0][mapping[i]] for i in range(1, 4)}
print('対応するHUBOの解:', hubo_configuration)
print('対応するHUBOの解のエネルギー:', dimod.BinaryPolynomial(polynomial, "BINARY").energy(hubo_configuration))

対応するHUBOの解: {1: 1, 2: 1, 3: 0}
対応するHUBOの解のエネルギー: -1.0


## 元のHUBOの最適解を確認

In [7]:
# 元のHUBOの最適解を確認する。
sampleset = dimod.ExactPolySolver().sample_hubo(H=polynomial)
print('最適解:', sampleset.first.sample)
print('対応するエネルギー:',sampleset.first.energy)

最適解: {1: 0, 2: 0, 3: 1}
対応するエネルギー: -5.0


## 考察
- HUBOの直接解法, QUBO変換による解法ともに、最適解と一致しなかった。

## 引用文献

-[1]Finding low-energy conformations of lattice protein models by quantum annealing Alejandro Perdomo-Ortiz, Neil Dickson, Marshall Drew-Brook, Geordie Rose & Alán Aspuru-Guzik Scientific Reports volume 2, Article number: 571 (2012)
- https://www.nature.com/articles/srep00571